#랜덤포레스트 다중분류
- wine dataset
- 포도주의 등급(퀄리티)를 예측하는 문제

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

##1.데이터 준비

In [3]:
!wget https://raw.githubusercontent.com/devdio/datasets/main/winequalityN.csv

--2023-11-30 12:13:05--  https://raw.githubusercontent.com/devdio/datasets/main/winequalityN.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 390376 (381K) [text/plain]
Saving to: ‘winequalityN.csv’

winequalityN.csv    100%[===================>] 381.23K  2.37MB/s    in 0.2s    

2023-11-30 12:13:07 (2.37 MB/s) - ‘winequalityN.csv’ saved [390376/390376]



In [4]:
wine = pd.read_csv('winequalityN.csv')
wine.shape

(6497, 13)

In [5]:
df = wine

In [6]:
df.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


##2.데이터 탐색

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  6497 non-null   object 
 1   fixed acidity         6487 non-null   float64
 2   volatile acidity      6489 non-null   float64
 3   citric acid           6494 non-null   float64
 4   residual sugar        6495 non-null   float64
 5   chlorides             6495 non-null   float64
 6   free sulfur dioxide   6497 non-null   float64
 7   total sulfur dioxide  6497 non-null   float64
 8   density               6497 non-null   float64
 9   pH                    6488 non-null   float64
 10  sulphates             6493 non-null   float64
 11  alcohol               6497 non-null   float64
 12  quality               6497 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB


In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
fixed acidity,6487.0,7.216579,1.296750,3.80000,6.40000,7.00000,7.70000,15.90000
volatile acidity,6489.0,0.339691,0.164649,0.08000,0.23000,0.29000,0.40000,1.58000
citric acid,6494.0,0.318722,0.145265,0.00000,0.25000,0.31000,0.39000,1.66000
residual sugar,6495.0,5.444326,4.758125,0.60000,1.80000,3.00000,8.10000,65.80000
chlorides,6495.0,0.056042,0.035036,0.00900,0.03800,0.04700,0.06500,0.61100
free sulfur dioxide,6497.0,30.525319,17.749400,1.00000,17.00000,29.00000,41.00000,289.00000
total sulfur dioxide,6497.0,115.744574,56.521855,6.00000,77.00000,118.00000,156.00000,440.00000
density,6497.0,0.994697,0.002999,0.98711,0.99234,0.99489,0.99699,1.03898
pH,6488.0,3.218395,0.160748,2.72000,3.11000,3.21000,3.32000,4.01000
sulphates,6493.0,0.531215,0.148814,0.22000,0.43000,0.51000,0.60000,2.00000


In [9]:
df['type'].value_counts()

white    4898
red      1599
Name: type, dtype: int64

In [10]:
df['quality'].value_counts()    #순서대로 안되있어 보기 힘듬

6    2836
5    2138
7    1079
4     216
8     193
3      30
9       5
Name: quality, dtype: int64

In [11]:
df['quality'].value_counts().sort_index()    #퀄리티는 타겟값인데 3~9로 넣는거 보다 순서대로 0~6으로 바꿔서 넣는게 좋음

3      30
4     216
5    2138
6    2836
7    1079
8     193
9       5
Name: quality, dtype: int64

In [12]:
df['quality'] = df['quality']-3

In [13]:
df['quality'].value_counts().sort_index()

0      30
1     216
2    2138
3    2836
4    1079
5     193
6       5
Name: quality, dtype: int64

#####결측치

In [14]:
df.isna().sum(axis=0)                  #결측치 제일많은게 종목당 10개여서 전체자료가 6000개여서 삭제해도 이상없을듯

type                     0
fixed acidity           10
volatile acidity         8
citric acid              3
residual sugar           2
chlorides                2
free sulfur dioxide      0
total sulfur dioxide     0
density                  0
pH                       9
sulphates                4
alcohol                  0
quality                  0
dtype: int64

In [15]:
df.dropna(inplace=True)

In [16]:
df.isna().sum(axis=0)

type                    0
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

##3.테스트 데이터 만들기

In [17]:
X= df.drop("quality",axis=1)
Y= df['quality']

In [18]:
# X=df.iloc[:,:-1]
# Y=df.iloc[:,-1]                 #위셀이랑 같은거임

In [19]:
#테스트 데이터 분리
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(X,Y, test_size=0.3, stratify=Y,random_state=0)     # stratify=Y는 비율로 잘나눠주는것

In [20]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((4524, 12), (1939, 12), (4524,), (1939,))

In [21]:
Y_train.value_counts().sort_index

<bound method Series.sort_index of 3    1974
2    1490
4     752
1     150
5     134
0      21
6       3
Name: quality, dtype: int64>

In [22]:
Y_test.value_counts().sort_index

<bound method Series.sort_index of 3    846
2    638
4    322
1     64
5     58
0      9
6      2
Name: quality, dtype: int64>

##4.데이터 전처리

#####오버샘플링

In [23]:
# !pip install -U imbalanced-learn

In [24]:
from imblearn.over_sampling import RandomOverSampler

ros =RandomOverSampler()
X_train, Y_train=ros.fit_resample(X_train,Y_train)

In [25]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((13818, 12), (1939, 12), (13818,), (1939,))

In [26]:
Y_train.value_counts().sort_index

<bound method Series.sort_index of 2    1974
3    1974
4    1974
1    1974
5    1974
0    1974
6    1974
Name: quality, dtype: int64>

In [27]:
X_train.head(3)

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,red,8.5,0.340,0.44,1.7,0.079,6.0,12.0,0.99605,3.52,0.63,10.7
1,white,7.8,0.300,0.37,1.3,0.051,16.0,96.0,0.99410,3.32,0.62,10.0
2,red,7.7,0.835,0.00,2.6,0.081,6.0,14.0,0.99750,3.30,0.52,9.3


#####레이블 인코딩

In [28]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X_train['type']=le.fit_transform(X_train['type'])
X_train

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,0,8.5,0.340,0.44,1.7,0.079,6.0,12.0,0.99605,3.52,0.63,10.7
1,1,7.8,0.300,0.37,1.3,0.051,16.0,96.0,0.99410,3.32,0.62,10.0
2,0,7.7,0.835,0.00,2.6,0.081,6.0,14.0,0.99750,3.30,0.52,9.3
3,1,6.2,0.250,0.47,11.6,0.048,62.0,210.0,0.99680,3.19,0.50,9.5
4,1,6.8,0.290,0.50,13.3,0.053,48.0,194.0,0.99740,3.09,0.45,9.4
...,...,...,...,...,...,...,...,...,...,...,...,...
13813,1,7.1,0.260,0.49,2.2,0.032,31.0,113.0,0.99030,3.37,0.42,12.9
13814,1,7.1,0.260,0.49,2.2,0.032,31.0,113.0,0.99030,3.37,0.42,12.9
13815,1,9.1,0.270,0.45,10.6,0.035,28.0,124.0,0.99700,3.20,0.46,10.4
13816,1,7.1,0.260,0.49,2.2,0.032,31.0,113.0,0.99030,3.37,0.42,12.9


#####스케일링

In [29]:
from sklearn.preprocessing import StandardScaler

ss=StandardScaler()
X_train_s=ss.fit_transform(X_train)
X_train_s[:5]                # 넘파이 어레이여서 .head() 를 못쓰고 [:4]이런거 써야함

array([[-1.95525872,  0.9000387 , -0.11929557,  0.75719751, -0.75431073,
         0.68850739, -1.09759726, -1.68900288,  0.60974079,  1.69376953,
         0.85082573, -0.09956225],
       [ 0.51144127,  0.36141375, -0.31762933,  0.28878388, -0.84747728,
        -0.06400043, -0.64147198, -0.31592333, -0.01982476,  0.47611975,
         0.77858966, -0.6360031 ],
       [-1.95525872,  0.28446732,  2.33508464, -2.18711673, -0.544686  ,
         0.74225795, -1.09759726, -1.65631051,  1.07787928,  0.35435477,
         0.05622892, -1.17244396],
       [ 0.51144127, -0.869729  , -0.56554652,  0.95794621,  1.55156131,
        -0.14462627,  1.45670431,  1.54754178,  0.85188139, -0.31535261,
        -0.08824323, -1.01917515],
       [ 0.51144127, -0.40805047, -0.36721277,  1.15869491,  1.94751914,
        -0.01024987,  0.81812892,  1.28600282,  1.04559387, -0.9241775 ,
        -0.4494236 , -1.09580955]])

In [30]:
type(X_train_s),type(Y_train)             #Y 넘파이 어레이로 바꾸기

(numpy.ndarray, pandas.core.series.Series)

In [31]:
Y_train=Y_train.values

In [32]:
type(Y_train)

numpy.ndarray

##5.학습

In [33]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier()
clf.fit(X_train_s,Y_train)


RandomForestClassifier()

##6.예측 및 성능 평가

In [34]:
X_test['type']= le.transform(X_test['type'])
X_test_s =ss.transform(X_test)
X_test_s[:4]

array([[ 0.51144127, -0.48499689, -0.11929557,  0.69028128, -0.77760237,
        -0.33275322, -0.04850911, -0.021692  , -1.29832711, -0.07182266,
        -0.52165967,  1.35649151],
       [ 0.51144127, -0.71583616, -0.56554652, -0.04579728,  0.13077147,
        -0.11775099,  0.49884122,  0.99177148,  0.27074396, -0.31535261,
         1.2120061 , -1.24907837],
       [ 0.51144127, -0.17721121, -0.91263059,  0.62336505, -0.82418564,
        -0.22525211,  0.77251639,  0.68119397, -0.82696008, -1.77653235,
        -1.46072864, -0.25283106],
       [ 0.51144127,  1.82339576, -0.61512996,  1.09177868, -0.00897835,
        -0.57463074, -1.23443484, -1.59092577, -0.15542349, -2.38535724,
         0.20070107, -0.6360031 ]])

In [35]:
Y_test=Y_test.values

In [36]:
Y_pred = clf.predict(X_test_s)

#####평가 메트릭스

In [37]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
def print_metrics(Y_test,Y_pred):
  acc= accuracy_score(Y_test,Y_pred)
  re= recall_score(Y_test,Y_pred,average='macro')
  pr= precision_score(Y_test,Y_pred,average='macro')
  f1= f1_score(Y_test,Y_pred,average='macro')
  print('정확도:',acc)
  print('재현율:',re)
  print('정밀도:',pr)
  print('f1스코어:',f1)
print_metrics(Y_test,Y_pred)

정확도: 0.6931407942238267
재현율: 0.40628043861478913
정밀도: 0.5520623026162409
f1스코어: 0.44635934058415133


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##7.튜닝

#####그리드 서치

In [38]:
from sklearn.model_selection import GridSearchCV

grid_params = {
    "max_depth":[13,16,19],
    'n_estimators':[80,100,120]
}

In [39]:
gs = GridSearchCV(clf, grid_params, cv=5,verbose=5)
gs.fit(X_train_s, Y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END .....max_depth=13, n_estimators=80;, score=0.893 total time=   3.1s
[CV 2/5] END .....max_depth=13, n_estimators=80;, score=0.892 total time=   3.4s
[CV 3/5] END .....max_depth=13, n_estimators=80;, score=0.901 total time=   2.1s
[CV 4/5] END .....max_depth=13, n_estimators=80;, score=0.904 total time=   1.1s
[CV 5/5] END .....max_depth=13, n_estimators=80;, score=0.923 total time=   1.1s
[CV 1/5] END ....max_depth=13, n_estimators=100;, score=0.890 total time=   1.4s
[CV 2/5] END ....max_depth=13, n_estimators=100;, score=0.889 total time=   1.4s
[CV 3/5] END ....max_depth=13, n_estimators=100;, score=0.907 total time=   1.4s
[CV 4/5] END ....max_depth=13, n_estimators=100;, score=0.908 total time=   2.0s
[CV 5/5] END ....max_depth=13, n_estimators=100;, score=0.927 total time=   2.1s
[CV 1/5] END ....max_depth=13, n_estimators=120;, score=0.892 total time=   1.8s
[CV 2/5] END ....max_depth=13, n_estimators=120;,

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [13, 16, 19],
                         'n_estimators': [80, 100, 120]},
             verbose=5)

In [40]:
gs.best_params_

{'max_depth': 19, 'n_estimators': 120}

In [41]:
gs.best_score_

0.9238698472108957

##8.최종 학습

In [42]:
clf = RandomForestClassifier(max_depth= 19, n_estimators= 120)
clf.fit(X_train_s,Y_train)

RandomForestClassifier(max_depth=19, n_estimators=120)

##9.모델 저장

In [43]:
import pickle

with open('rf-wine-model.pkl','wb') as file:
  pickle.dump(clf,file)

In [44]:
with open('rf-wine-model.pkl','wb') as file:
  pickle.dump(clf,file)

In [45]:
with open('tree-penguins-le.pkl','wb') as file:
  pickle.dump(clf,file)